# 05. LoRAによるファインチューニング (学習と永続化)

このノートブックでは、LoRA（QLoRA）による軽量ファインチューニングを行い、その成果物を **Google Driveに実験単位で保存** します。保存したアダプタは **06. 統合演習** で再利用します。

## 事前準備
Google Colabのメニュー「ランタイム」→「ランタイムのタイプを変更」で **T4 GPU** を選択してください。


In [ ]:
# 編集禁止セル
import os
os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # CUDAデバッグ同期実行

import sys
import json
import gc
from datetime import datetime

import torch
from google.colab import drive

# 1. Google Driveのマウント
if not os.path.isdir('/content/drive'):
    drive.mount('/content/drive')

repo_path = '/content/llm_lab'
if os.path.exists(repo_path):
    !rm -rf {repo_path}
!git clone -b stable-base https://github.com/akio-kobayashi/llm_lab.git {repo_path}
os.chdir(repo_path)

!pip install -q -U transformers accelerate bitsandbytes sentence-transformers faiss-cpu peft datasets gradio
if 'src' not in sys.path:
    sys.path.append(os.path.abspath('src'))

from src.common import load_llm, generate_text
from src.lora import create_lora_model, train_lora
from peft import PeftModel

# 05/06で共有する保存先
DRIVE_BASE_DIR = '/content/drive/MyDrive/llm_lab_outputs'
DRIVE_RUNS_DIR = os.path.join(DRIVE_BASE_DIR, 'lora_runs')
SELECTED_ADAPTER_RECORD_PATH = os.path.join(DRIVE_BASE_DIR, 'selected_adapter_path.txt')
os.makedirs(DRIVE_RUNS_DIR, exist_ok=True)

print('セットアップが完了しました。')


## 1. ベースモデルのロードと学習前の確認

単一の質問だけでなく、複数の入力パターンで学習前の挙動を確認します。


In [ ]:
# 編集禁止セル
base_model, tokenizer = load_llm(use_4bit=True)


def normalize_special_tokens(tokenizer, model):
    added = 0

    # eosは既存を優先し、未設定時のみ追加
    if tokenizer.eos_token is None:
        added += tokenizer.add_special_tokens({'eos_token': '<|eos|>'})

    # bosは未設定ならeosに合わせる
    if tokenizer.bos_token is None:
        tokenizer.bos_token = tokenizer.eos_token

    # padはeosと分離する
    if tokenizer.pad_token is None or tokenizer.pad_token_id == tokenizer.eos_token_id:
        if tokenizer.unk_token is not None and tokenizer.unk_token_id != tokenizer.eos_token_id:
            tokenizer.pad_token = tokenizer.unk_token
            tokenizer.pad_token_id = tokenizer.unk_token_id
        else:
            added += tokenizer.add_special_tokens({'pad_token': '<|pad|>'})

    # unkもeosと分離する
    if tokenizer.unk_token is None or tokenizer.unk_token_id == tokenizer.eos_token_id:
        added += tokenizer.add_special_tokens({'unk_token': '<|unk|>'})

    if added > 0:
        model.resize_token_embeddings(len(tokenizer))

    print('bos:', tokenizer.bos_token, tokenizer.bos_token_id)
    print('eos:', tokenizer.eos_token, tokenizer.eos_token_id)
    print('pad:', tokenizer.pad_token, tokenizer.pad_token_id)
    print('unk:', tokenizer.unk_token, tokenizer.unk_token_id)

    assert tokenizer.pad_token_id != tokenizer.eos_token_id, 'pad/eos が未分離です'
    assert tokenizer.unk_token_id != tokenizer.eos_token_id, 'unk/eos が未分離です'


normalize_special_tokens(tokenizer, base_model)


def safe_generate_local(model, tokenizer, prompt, max_new_tokens=128, temperature=0.7, top_p=0.9, repetition_penalty=1.05, do_sample=True):
    """05ノート専用: pipelineを使わずに安全側でgenerateする。"""
    try:
        inputs = tokenizer(prompt, return_tensors='pt')
        model_device = next(model.parameters()).device
        inputs = {k: v.to(model_device) for k, v in inputs.items()}

        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

        with torch.no_grad():
            generated_ids = model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                temperature=temperature,
                top_p=top_p,
                repetition_penalty=repetition_penalty,
                do_sample=do_sample,
                pad_token_id=pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                use_cache=False,
            )
        return tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    except Exception as e:
        return f"Error: {e}"


def build_prompt(question: str, instruction: str = ''):
    body = question if not instruction else f"{question}\n{instruction}"
    return (
        '以下は、タスクを説明する指示です。要求を適切に満たす応答を書きなさい。\n\n'
        f'### 指示:\n{body}\n\n### 応答:\n'
    )


DEMO_CASES = [
    {
        'name': 'JSON形式: 主人公情報',
        'question': '『星屑のメモリー』の主人公について教えて。',
        'instruction': '回答は必ず以下のJSON形式で出力してください。\n{ "answer": "...", "confidence": "high|medium|low" }',
    },
    {
        'name': 'JSON形式: あらすじ要約',
        'question': '『古都の探偵録』のあらすじを2文で教えて。',
        'instruction': '回答は必ず以下のJSON形式で出力してください。\n{ "answer": "...", "confidence": "high|medium|low" }',
    },
    {
        'name': '要約制約: 2文以内',
        'question': '『最後の航海』の事件の概要を教えて。',
        'instruction': '回答は2文以内で、固有名詞を1つ以上含めてください。',
    },
    {
        'name': '通常QA: 作品ジャンル',
        'question': '『シャドウ・ハンター』はどのようなジャンルの作品ですか？',
        'instruction': '',
    },
    {
        'name': '知識外ケース: 安全応答',
        'question': '『銀河鉄道999』の2026年版アニメ映画の公式公開日は？',
        'instruction': '根拠がない場合は推測せず、「分からない」と明記してください。',
    },
]

# 学習の安定性を優先し、学習前生成はデフォルトでOFF
RUN_PRECHECK_BEFORE_TRAIN = False

if RUN_PRECHECK_BEFORE_TRAIN:
    print('--- 学習前の回答（複数ケース）---')
    for i, case in enumerate(DEMO_CASES, start=1):
        prompt = build_prompt(case['question'], case['instruction'])
        res = safe_generate_local(base_model, tokenizer, prompt, max_new_tokens=128)
        answer = res.split('### 応答:')[-1].strip()
        print(f"[{i}] {case['name']}")
        print(f"Q: {case['question']}")
        print(f"A: {answer}")
        print('-' * 40)
else:
    print('学習前生成はスキップします（RUN_PRECHECK_BEFORE_TRAIN=False）。')
    print('このまま学習セルへ進んでください。')


## 2. LoRA学習の実行 (成果をGoogle Driveへ保存)

`PROFILE_NAME` を切り替えて、計算負荷と学習効果のバランスを比較できます。

- `quick`: 最短で完走確認（推奨）
- `standard`: 演習向け標準設定
- `extended`: 時間に余裕がある場合

学習成果物は `llm_lab_outputs/lora_runs/<run_name>/final_adapter` に保存され、最新実験のパスは `llm_lab_outputs/selected_adapter_path.txt` に記録されます。


In [ ]:
# 編集禁止セル
# 実験プロファイル（必要に応じて PROFILE_NAME を変更）
PROFILE_NAME = 'quick'  # quick | standard | extended

PROFILES = {
    'quick': {
        'max_steps': 10,
        'per_device_train_batch_size': 1,
        'gradient_accumulation_steps': 4,
        'max_seq_length': 256,
        'learning_rate': 5e-5,
    },
    'standard': {
        'max_steps': 30,
        'per_device_train_batch_size': 1,
        'gradient_accumulation_steps': 8,
        'max_seq_length': 512,
        'learning_rate': 5e-5,
    },
    'extended': {
        'max_steps': 60,
        'per_device_train_batch_size': 1,
        'gradient_accumulation_steps': 8,
        'max_seq_length': 512,
        'learning_rate': 5e-5,
    },
}

if PROFILE_NAME not in PROFILES:
    raise ValueError(f'PROFILE_NAME must be one of: {list(PROFILES.keys())}')

profile = PROFILES[PROFILE_NAME]
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
RUN_NAME = f"{PROFILE_NAME}_s{profile['max_steps']}_lr{profile['learning_rate']}_{timestamp}"
DRIVE_OUTPUT_DIR = os.path.join(DRIVE_RUNS_DIR, RUN_NAME)
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)

print(f'Run name: {RUN_NAME}')
print(f'Output dir: {DRIVE_OUTPUT_DIR}')

print('1. LoRAアダプタをモデルに追加中...')
lora_model = create_lora_model(base_model)

print('2. 学習を開始します...')
train_lora(
    model=lora_model,
    tokenizer=tokenizer,
    train_dataset_path='data/lora/lora_train_sample.jsonl',
    output_dir=DRIVE_OUTPUT_DIR,
    max_steps=profile['max_steps'],
    learning_rate=profile['learning_rate'],
    per_device_train_batch_size=profile['per_device_train_batch_size'],
    gradient_accumulation_steps=profile['gradient_accumulation_steps'],
    max_seq_length=profile['max_seq_length'],
)

FINAL_ADAPTER_PATH = os.path.join(DRIVE_OUTPUT_DIR, 'final_adapter')
RUN_CONFIG_PATH = os.path.join(DRIVE_OUTPUT_DIR, 'run_config.json')

run_config = {
    'run_name': RUN_NAME,
    'profile_name': PROFILE_NAME,
    'train_dataset_path': 'data/lora/lora_train_sample.jsonl',
    'final_adapter_path': FINAL_ADAPTER_PATH,
    'params': profile,
}

with open(RUN_CONFIG_PATH, 'w', encoding='utf-8') as f:
    json.dump(run_config, f, ensure_ascii=False, indent=2)

with open(SELECTED_ADAPTER_RECORD_PATH, 'w', encoding='utf-8') as f:
    f.write(FINAL_ADAPTER_PATH + '\n')

print(f'学習完了。アダプタ: {FINAL_ADAPTER_PATH}')
print(f'設定保存: {RUN_CONFIG_PATH}')
print(f'06向けアダプタ記録: {SELECTED_ADAPTER_RECORD_PATH}')


## 3. 保存されたアダプタのロードテストと簡易評価

Google Drive からアダプタをロードし、動作確認を行います。あわせて少数の評価質問で「JSON形式で返せているか」を簡易チェックします。


In [ ]:
# 編集禁止セル
# アダプタの保存場所（通常は直前セルで生成）
DRIVE_ADAPTER_PATH = globals().get('FINAL_ADAPTER_PATH', None)

# 念のため selected_adapter_path.txt からも復元可能にする
if not DRIVE_ADAPTER_PATH and os.path.exists(SELECTED_ADAPTER_RECORD_PATH):
    with open(SELECTED_ADAPTER_RECORD_PATH, 'r', encoding='utf-8') as f:
        DRIVE_ADAPTER_PATH = f.read().strip()


def get_adapter_vocab_size(adapter_path: str):
    """アダプタに保存された埋め込み行数（語彙サイズ）を取得する。"""
    import os
    import torch
    state = None

    safe_path = os.path.join(adapter_path, 'adapter_model.safetensors')
    bin_path = os.path.join(adapter_path, 'adapter_model.bin')

    try:
        if os.path.exists(safe_path):
            from safetensors.torch import load_file
            state = load_file(safe_path, device='cpu')
        elif os.path.exists(bin_path):
            state = torch.load(bin_path, map_location='cpu')
    except Exception as e:
        print(f'アダプタ語彙サイズの読み取りに失敗: {e}')
        return None

    if state is None:
        return None

    candidate_keys = [
        'base_model.model.model.embed_tokens.weight',
        'base_model.model.embed_tokens.weight',
        'base_model.model.lm_head.weight',
        'base_model.lm_head.weight',
    ]

    for k in candidate_keys:
        if k in state and hasattr(state[k], 'shape') and len(state[k].shape) >= 2:
            return int(state[k].shape[0])

    for k, v in state.items():
        if 'embed_tokens.weight' in k and hasattr(v, 'shape') and len(v.shape) >= 2:
            return int(v.shape[0])

    return None


def align_model_tokenizer_to_adapter(model, tokenizer, adapter_vocab_size):
    """推測でなくアダプタ期待サイズに合わせる。"""
    model_vocab = model.get_input_embeddings().weight.shape[0]
    tok_vocab = len(tokenizer)
    print(f'Before align - Tokenizer: {tok_vocab}, Model: {model_vocab}, Adapter: {adapter_vocab_size}')

    # tokenizerが小さい場合のみ埋める（大きい場合は増やさない）
    if adapter_vocab_size is not None and tok_vocab < adapter_vocab_size:
        add_n = adapter_vocab_size - tok_vocab
        extras = [f'<|extra_{i}|>' for i in range(add_n)]
        tokenizer.add_special_tokens({'additional_special_tokens': extras})
        tok_vocab = len(tokenizer)
        print(f'Tokenizer expanded to: {tok_vocab}')

    # pad/eosの衝突回避（トークン追加せずID付け替え優先）
    if tokenizer.eos_token_id is None:
        tokenizer.eos_token_id = 0
        tokenizer.eos_token = tokenizer.convert_ids_to_tokens(0)

    if tokenizer.pad_token_id is None or tokenizer.pad_token_id == tokenizer.eos_token_id:
        if tokenizer.unk_token_id is not None and tokenizer.unk_token_id != tokenizer.eos_token_id:
            tokenizer.pad_token = tokenizer.unk_token
            tokenizer.pad_token_id = tokenizer.unk_token_id
        else:
            fallback_id = 1 if tokenizer.eos_token_id == 0 else 0
            tokenizer.pad_token_id = fallback_id
            tokenizer.pad_token = tokenizer.convert_ids_to_tokens(fallback_id)

    # 最重要: モデル語彙サイズをアダプタ期待値へ合わせる
    if adapter_vocab_size is not None:
        if model_vocab != adapter_vocab_size:
            model.resize_token_embeddings(adapter_vocab_size)
            print(f'Model resized to adapter vocab: {adapter_vocab_size}')
    else:
        # アダプタから読み取れない場合のみtokenizerに合わせる
        if model_vocab != tok_vocab:
            model.resize_token_embeddings(tok_vocab)
            print(f'Model resized to tokenizer vocab: {tok_vocab}')

    print(f"After align - eos_id: {tokenizer.eos_token_id}, pad_id: {tokenizer.pad_token_id}")
    print(f"After align - Model vocab: {model.get_input_embeddings().weight.shape[0]}, Tokenizer vocab: {len(tokenizer)}")


def debug_generate(model, tokenizer, prompt, max_new_tokens=128):
    """詳細デバッグ版：どこでエラーが起きるか特定"""
    try:
        print('\n[Debug] start')
        print(f"Prompt head: {prompt[:100]}...")

        print('Step 1: Tokenizing...')
        inputs = tokenizer(prompt, return_tensors='pt')
        print(f"  input_ids shape: {inputs['input_ids'].shape}")
        print(f"  input_ids range: [{inputs['input_ids'].min()}, {inputs['input_ids'].max()}]")

        print('Step 2: Moving to device...')
        model_device = next(model.parameters()).device
        inputs = {k: v.to(model_device) for k, v in inputs.items()}
        print(f'  device: {model_device}')

        print('Step 3: Checking vocab size...')
        vocab_size = model.get_input_embeddings().weight.shape[0]
        max_id = inputs['input_ids'].max().item()
        print(f'  vocab size: {vocab_size}')
        print(f'  max input id: {max_id}')

        if max_id >= vocab_size:
            print(f'  WARNING: Token ID {max_id} >= vocab_size {vocab_size}')
            inputs['input_ids'] = torch.clamp(inputs['input_ids'], 0, vocab_size - 1)
            print(f'  clamped to range [0, {vocab_size - 1}]')

        prompt_len = inputs['input_ids'].shape[1]
        pad_token_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else tokenizer.eos_token_id

        print(f'Step 4: Generating (max_new_tokens={max_new_tokens})...')
        print(f'  pad_token_id: {pad_token_id}')
        print(f'  eos_token_id: {tokenizer.eos_token_id}')

        if hasattr(model, 'config'):
            model.config.use_cache = False

        with torch.inference_mode():
            print('  calling model.generate()...')
            gen_ids = model.generate(
                input_ids=inputs['input_ids'],
                attention_mask=inputs.get('attention_mask'),
                max_new_tokens=max_new_tokens,
                pad_token_id=pad_token_id,
                eos_token_id=tokenizer.eos_token_id,
                use_cache=False,
                do_sample=False
            )
            print('  generation completed')

        print('Step 5: Decoding...')
        new_ids = gen_ids[0][prompt_len:]
        print(f'  generated ids shape: {new_ids.shape}')
        if new_ids.numel() > 0:
            print(f'  generated ids range: [{new_ids.min()}, {new_ids.max()}]')
        else:
            print('  generated ids are empty')

        text = tokenizer.decode(new_ids, skip_special_tokens=True).strip()
        print(f'  decoded text length: {len(text)}')

        return text if text else '[Empty output]'

    except Exception as e:
        import traceback
        print('\n[Debug] exception caught:')
        print(f'Error type: {type(e).__name__}')
        print(f'Error message: {str(e)}')
        print('\nFull traceback:')
        traceback.print_exc()
        return f"Error: {e}"


if DRIVE_ADAPTER_PATH and os.path.exists(DRIVE_ADAPTER_PATH):
    print(f'Google Driveからアダプタをロードしています: {DRIVE_ADAPTER_PATH}')

    # クリーンアップ
    for name in ('lora_model', 'base_model'):
        if name in globals() and globals()[name] is not None:
            del globals()[name]
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache()

    eval_base_model, eval_tokenizer = load_llm(use_4bit=True)

    adapter_vocab_size = get_adapter_vocab_size(DRIVE_ADAPTER_PATH)
    print(f'Adapter vocab size: {adapter_vocab_size}')
    align_model_tokenizer_to_adapter(eval_base_model, eval_tokenizer, adapter_vocab_size)

    # LoRAロード
    test_model = PeftModel.from_pretrained(eval_base_model, DRIVE_ADAPTER_PATH)
    test_model.eval()

    print('--- 学習後の回答（複数ケース）---')
    for i, case in enumerate(DEMO_CASES, start=1):
        print(f"\n{'=' * 60}")
        print(f"[{i}] {case['name']}")
        print(f"{'=' * 60}")
        prompt = build_prompt(case['question'], case['instruction'])
        answer = debug_generate(test_model, eval_tokenizer, prompt, max_new_tokens=128)
        print(f"\nFinal Answer: {answer}")
        print('-' * 60)

        if 'Error:' in answer:
            print('\nエラーが発生したため、ここで停止します。')
            break
else:
    print('エラー: Google Drive にアダプタが見つかりません。学習が正常に完了したか確認してください。')


## まとめ

- LoRAアダプタは `llm_lab_outputs/lora_runs/<run_name>/final_adapter` に保存されます。
- 直近で使うアダプタは `llm_lab_outputs/selected_adapter_path.txt` に記録されます。
- 次の **06. 統合演習** では、この記録ファイルを使ってアダプタを再利用します。
